# Topic modelling

Applying Gensim (Topic Modeling Library): 
Gensim is a library for topic modeling and document similarity analysis, which can be used to identify similarities and patterns in large volumes of text. By analyzing the similarities between documents and topics, Gensim can help to identify potential cases of disinformation and trace their sources.

In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import pickle
from IPython.core.display import display, HTML
from tqdm.notebook import tqdm
import glob
import csv


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from matplotlib import pylab as plt
%matplotlib inline

pd.set_option("display.max_rows", 2000)
pd.set_option("display.max_colwidth", 5000)
pd.set_option("display.max_columns", 2000)


DATA_DIR = '../data/'

/var/folders/zh/c52bnb1524xfzk_885ccgt4w0000gq/T/ipykernel_32215/3228527578.py:5: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [4]:
glob.glob(f"{DATA_DIR}*")

['../data/NYT_Russia_Ukraine.csv',
 '../data/Ukraine_troops.csv',
 '../data/Guardians_Russia_Ukraine.csv',
 '../data/Ukraine_war.csv',
 '../data/Russia_invade.csv',
 '../data/russian_invasion_of_ukraine.csv',
 '../data/Ukraine_nato.csv',
 '../data/Russian_troops.csv',
 '../data/StandWithUkraine.csv',
 '../data/Ukraine_border.csv',
 '../data/Russian_border_Ukraine.csv']

# Load data

In [5]:
import pickle

data_tweet_clean = pickle.load(open('data_clean.pkl', 'rb'))
data_tweet_clean.shape

(787029, 31)

# Text preprocessing

In [6]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [7]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# Convert to list
data = data_tweet_clean.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['Hugs to all Hope this is spread to Russian and Ukraine troops #Ukraine '
 '#NoWar #Stop_the_War #We_are_all_friends #Wake_up_troops #Hugs_to_all '
 'https://t.co/XFahrOAQnO']


In [8]:
import string

def strip_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text

def strip_all_entities(text):
    entity_prefixes = ['@','#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)


#remove hashtags
data = [strip_all_entities(strip_links(sent)) for sent in data]

pprint(data[:1])

['Hugs to all Hope this is spread to Russian and Ukraine troops the War are '
 'all friends up troops to all']


In [9]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['hugs', 'to', 'all', 'hope', 'this', 'is', 'spread', 'to', 'russian', 'and', 'ukraine', 'troops', 'the', 'war', 'are', 'all', 'friends', 'up', 'troops', 'to', 'all']]


In [10]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['hugs', 'to', 'all', 'hope', 'this', 'is', 'spread', 'to', 'russian', 'and', 'ukraine', 'troops', 'the', 'war', 'are', 'all', 'friends', 'up', 'troops', 'to', 'all']


In [11]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [12]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['hug', 'hope', 'spread', 'troop', 'war', 'friend', 'troop']]


In [13]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1)]]


# Build 20 topics extraction model

In [17]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.381*"border" + 0.028*"war" + 0.025*"cross" + 0.025*"invade" + '
  '0.024*"talk" + 0.018*"missile" + 0.015*"send" + 0.015*"report" + '
  '0.014*"build" + 0.012*"put"'),
 (1,
  '0.035*"western" + 0.035*"new" + 0.031*"support" + 0.028*"defend" + '
  '0.028*"open" + 0.027*"good" + 0.026*"territory" + 0.025*"hold" + '
  '0.024*"trump" + 0.019*"stand"'),
 (2,
  '0.216*"russian" + 0.066*"ukraine" + 0.039*"troop" + 0.038*"military" + '
  '0.036*"force" + 0.030*"ukrainian" + 0.019*"country" + 0.018*"go" + '
  '0.015*"move" + 0.013*"take"'),
 (3,
  '0.057*"tank" + 0.048*"close" + 0.038*"crisis" + 0.031*"official" + '
  '0.024*"let" + 0.024*"kyiv" + 0.023*"break" + 0.022*"continue" + '
  '0.022*"already" + 0.020*"news"'),
 (4,
  '0.154*"want" + 0.057*"southern" + 0.048*"sanction" + 0.043*"allow" + '
  '0.035*"amp" + 0.028*"land" + 0.026*"control" + 0.025*"really" + '
  '0.023*"promise" + 0.020*"sure"'),
 (5,
  '0.073*"attack" + 0.035*"right" + 0.031*"unit" + 0.030*"side" + 0.030*"part" 

In [18]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

# Perplexity:  -18.273007475627555

# Coherence Score:  0.24473090500373823


Perplexity:  -7.951003842343718

Coherence Score:  0.2751166354661973


In [19]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/dzikr/mambaforge/envs/research/lib/python3.10/site-packages/pyLDAvis/_prepare.py:244: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.309440  0.311615       1        1  31.182996
0      0.281818 -0.360357       2        1  19.031608
6      0.031356  0.064522       3        1  11.659859
5     -0.029073 -0.014058       4        1   7.483618
3     -0.049725 -0.018929       5        1   7.104158
1     -0.067741  0.075758       6        1   5.831936
8     -0.114221 -0.007219       7        1   4.734496
7     -0.122311 -0.008001       8        1   4.567298
4     -0.113857 -0.030547       9        1   4.563323
9     -0.125687 -0.012784      10        1   3.840709, topic_info=               Term           Freq          Total Category  logprob  loglift
346          border  641383.000000  641383.000000  Default  30.0000  30.0000
16          russian  595728.000000  595728.000000  Default  29.0000  29.0000
70          ukraine  182781.000000  182781.000000  Default  28.0000  28.0000
44              say   93614.000000   93614.000000  Default  27.0000  27.0000
41             want   62009.000000   62009.000000  Default  26.0000  26.0000
43         invasion   70587.000000   70587.000000  Default  25.0000  25.0000
4             troop  108070.000000  108070.000000  Default  24.0000  24.0000
542          attack   48356.000000   48356.000000  Default  23.0000  23.0000
388        military  104111.000000  104111.000000  Default  22.0000  22.0000
8             force   99093.000000   99093.000000  Default  21.0000  21.0000
703            show   32322.000000   32322.000000  Default  20.0000  20.0000
18             tank   35970.000000   35970.000000  Default  19.0000  19.0000
33        ukrainian   81531.000000   81531.000000  Default  18.0000  18.0000
149               s   28246.000000   28246.000000  Default  17.0000  17.0000
2227          guard   27788.000000   27788.000000  Default  16.0000  16.0000
654           close   30407.000000   30407.000000  Default  15.0000  15.0000
379            know   25939.000000   25939.000000  Default  14.0000  14.0000
60              get   36211.000000   36211.000000  Default  13.0000  13.0000
5               war   46593.000000   46593.000000  Default  12.0000  12.0000
1058         region   23619.000000   23619.000000  Default  11.0000  11.0000
1694       southern   23217.000000   23217.000000  Default  10.0000  10.0000
543           cross   42492.000000   42492.000000  Default   9.0000   9.0000
37           invade   41688.000000   41688.000000  Default   8.0000   8.0000
875         belarus   31678.000000   31678.000000  Default   7.0000   7.0000
39             talk   40708.000000   40708.000000  Default   6.0000   6.0000
87            biden   21706.000000   21706.000000  Default   5.0000   5.0000
780          crisis   23683.000000   23683.000000  Default   4.0000   4.0000
235           right   23368.000000   23368.000000  Default   3.0000   3.0000
168         country   51246.000000   51246.000000  Default   2.0000   2.0000
115           video   18230.000000   18230.000000  Default   1.0000   1.0000
16          russian  595727.694043  595728.547118   Topic1  -1.5331   1.1653
70          ukraine  182780.731917  182781.584916   Topic1  -2.7146   1.1653
4             troop  108069.986757  108070.839822   Topic1  -3.2401   1.1653
388        military  104110.230409  104111.083483   Topic1  -3.2775   1.1653
8             force   99092.720645   99093.573686   Topic1  -3.3269   1.1653
33        ukrainian   81530.531732   81531.384731   Topic1  -3.5219   1.1653
168         country   51245.573951   51246.426919   Topic1  -3.9863   1.1653
80               go   49425.937456   49426.790424   Topic1  -4.0224   1.1653
66             move   40017.411388   40018.264442   Topic1  -4.2336   1.1653
52             take   37176.033703   37176.886693   Topic1  -4.3073   1.1653
111          people   33196.421924   33197.274878   Topic1  -4.4205   1.1653
221         soldier   31025.438120   31026.291133   Topic1  -4.4881   1.165